한국어 형태소 분석기 간 차이 
- https://mr-doosun.tistory.com/22
- https://soohee410.github.io/compare_tagger 
- https://konlpy-ko.readthedocs.io/ko/v0.4.3/api/konlpy.tag/

특징적인 차이:
- mecab 속도가 빠름 
- kkma 속도가 가장 느림 
- okt는 어근 추출에 뛰어남 (ex. 가는, 가던, 가고 == 가다 로 추출)

konlpy 이외의 한국어 전처리 패키지 
https://mangastorytelling.tistory.com/entry/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84%EC%9D%B4%EC%9A%A9%ED%95%9C-%EC%9E%90%EC%97%B0%EC%96%B4-%EC%B2%98%EB%A6%AC-%EC%9E%85%EB%AC%B8-0210-%ED%95%9C%EA%B5%AD%EC%96%B4-%EC%A0%84%EC%B2%98%EB%A6%AC-%ED%8C%A8%ED%82%A4%EC%A7%80-Text-Preprocessing-Tools-for-Korean-Text

In [1]:
import pandas as pd
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 100
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 텍스트 전처리
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from konlpy.tag import Okt 
from ckonlpy.tag import Twitter
import re 
from string import punctuation
import requests
import pickle
import ast

# 토픽모델링
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis 
import pyLDAvis.gensim

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


#### 미리 만들어둔 리뷰데이터 불러오기 

In [7]:
df = pd.read_csv('D:/review_df_.csv')

df.head(3)

,Unnamed: 0,DATE,STAR,REVIEW,LIKE
0,0,2020-01-07,1,오류거지같음 쓸때마다 앱자체 오류도 많이 나고 이용자수 많다면서 결제도 안되고 툭하...,0
1,1,2020-01-07,5,국굿 아이폰 바코드나 큐알코드 결제만 앱에서 가능 하면 최고일것 같아요,0
2,2,2020-01-07,3,어플 팅김현상 2월달부터 잘 사용하다가 오늘 업데이트를 했는데 어플 접속이 안됩니다...,0


In [8]:
del df['Unnamed: 0'] 

df.head(3)

,DATE,STAR,REVIEW,LIKE
0,2020-01-07,1,오류거지같음 쓸때마다 앱자체 오류도 많이 나고 이용자수 많다면서 결제도 안되고 툭하...,0
1,2020-01-07,5,국굿 아이폰 바코드나 큐알코드 결제만 앱에서 가능 하면 최고일것 같아요,0
2,2020-01-07,3,어플 팅김현상 2월달부터 잘 사용하다가 오늘 업데이트를 했는데 어플 접속이 안됩니다...,0


In [5]:
df.shape # 총 2871개 리뷰

(2871, 4)

#### 4월 이전 리뷰만 떼어서 새로운 df 만들기

In [6]:
df_before = df[df['DATE'] <= '2021-04-04']

df_before.shape

(1856, 4)

In [6]:
df_before.head(8)

,DATE,STAR,REVIEW,LIKE
0,2020-01-07,1,오류거지같음 쓸때마다 앱자체 오류도 많이 나고 이용자수 많다면서 결제도 안되고 툭하...,0
1,2020-01-07,5,국굿 아이폰 바코드나 큐알코드 결제만 앱에서 가능 하면 최고일것 같아요,0
2,2020-01-07,3,어플 팅김현상 2월달부터 잘 사용하다가 오늘 업데이트를 했는데 어플 접속이 안됩니다...,0
3,2020-01-07,2,"동백전 어플설치 저는 아이폰이라 설치를 했지만, 아버지도 동백전 어플 설치해드리려고...",0
4,2020-01-07,4,개선사항 요청하고 싶습니다! 본인 인증 수단을 공인인증서도 추가 하였으면 좋겠습니다...,0
5,2020-01-07,1,ㅠ 주민등록증 찍는데 청소년들은 청소년증으로 하라해서 청소년증 찍고 발급날짜 했는데...,0
6,2020-01-07,1,불안정한 앱 이용내역 확인 시 강제종료 앱 메뉴도 간단하고 동백전 서비스도 훌륭한데,0
7,2020-01-07,1,앱 꺼짐 문제 앱 들어가서 캐쉬백을 누르면 앱이 꺼져버리는데 어떻게 해야되나요? 다...,0


In [7]:
# 토픽모델링에서의 train set, test set 분리를 위해 미리 데이터 셔플링

df_before = df_before.sample(frac=1).reset_index(drop=True)

df_before.head(8)

,DATE,STAR,REVIEW,LIKE
0,2020-05-13,1,업데이트도안되고 새로 지우고 설치하는데 설치가 안됩니다.,0
1,2020-03-30,2,오늘 하루종일 먹통에 2중 결재 됐습니다 확인해주세요,0
2,2020-04-30,5,좋아요,0
3,2020-04-10,1,와파든 데이터든 안들어가져요,0
4,2020-04-16,1,카드취소후 동백전금액이 회복이 안되어 문의할래도 하루종일 전화연결은 안되고 지난번 ...,1
5,2020-04-13,5,만족함..,1
6,2020-03-10,1,가입이 안됩니다!!인증이 안되요~ㅎㅎ발급 받아보려고 어플깔고 휴대폰 인증하는거 아무...,1
7,2020-03-03,5,좋아요,0


#### corpus (말뭉치) 생성

In [9]:
# 각 행들을 corpus에 저장 

corpus = df['REVIEW']

corpus

0       오류거지같음 쓸때마다 앱자체 오류도 많이 나고 이용자수 많다면서 결제도 안되고 툭하...
1                 국굿 아이폰 바코드나 큐알코드 결제만 앱에서 가능 하면 최고일것 같아요
2       어플 팅김현상 2월달부터 잘 사용하다가 오늘 업데이트를 했는데 어플 접속이 안됩니다...
3       동백전 어플설치 저는 아이폰이라 설치를 했지만, 아버지도 동백전 어플 설치해드리려고...
4       개선사항 요청하고 싶습니다! 본인 인증 수단을 공인인증서도 추가 하였으면 좋겠습니다...
5       ㅠ 주민등록증 찍는데 청소년들은 청소년증으로 하라해서 청소년증 찍고 발급날짜 했는데...
6         불안정한 앱 이용내역 확인 시 강제종료 앱 메뉴도 간단하고 동백전 서비스도 훌륭한데 
7       앱 꺼짐 문제 앱 들어가서 캐쉬백을 누르면 앱이 꺼져버리는데 어떻게 해야되나요? 다...
8                      안들어가져요 업데이트 했는데 안들어가져요.. 해결 좀 해주세여
9       고객 서비스가 최악이네요 어플 오류로 인하여 충전 하려고 하니 바탕화면으로 팅기는데...
10      왜때문에 인증이안되나효 내이름에 내 탄생일에 내 명의의 핸드폰번혼데 자꾸 확인하래 ...
11      동백점 미가맹점 결제 가능 요청 잘 사용중입니다. 감사합니다^^* 다만 동백점 미가...
12      카드신청안됨 다 하고 마지막에 계속 오류가 나요ㅠ 청구서 수령관련 등록을 해야된다는...
13                                 아이폰 X 충전안됨 충전하러 가면 튕겨요
14      오류 갑자기 오늘 접속오류 계속 뜨길래 삭제 후 다시 깔았더니 아예 안되네요; 빠른...
15       돈 넣었는데 어플 실행이 안돼요 ? 잘 쓰다가 돈 남아있는데 어플 실행이 안돼요 뭔데요
16                            503오류 뜨면서 실행안되네요 우째해야되나요ㅡ.ㅡ
17      갑자기 서비

#### tf-idf 활용한 빈도 분석 

In [9]:
total_tokens = [token for mag in corpus for token in str(mag).split()]

print('띄어쓰기 기준으로 나누어봤을 때의 단어(토큰)의 개수:', len(total_tokens), '개')

띄어쓰기 기준으로 나누어봤을 때의 단어(토큰)의 개수: 14699 개


In [10]:
total_tokens[:10]

['업데이트도안되고', '새로', '지우고', '설치하는데', '설치가', '안됩니다.', '오늘', '하루종일', '먹통에', '2중']

단순 빈도분석

In [11]:
tfidf = TfidfVectorizer(max_features=1000)
tdm = tfidf.fit_transform(corpus)

In [12]:
word_count = pd.DataFrame({'단어': tfidf.get_feature_names(), 
                           '빈도': tdm.sum(axis=0).flat})

In [13]:
word_count.sort_values('빈도', ascending=False).head(20)

,단어,빈도
799,좋아요,201.834814
138,너무,39.099957
200,동백전,35.820046
796,좋습니다,27.888702
85,계속,25.015618
810,좋음,21.356964
520,아주,20.725601
603,업데이트가,19.300822
861,최고,19.293602
754,접속이,18.745233


> TF-IDF으로 단어에 가중치 주었을 때의 결과가 달라짐을 알 수 있다. 

참고: https://hanawithdata.tistory.com/entry/%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EC%84%9D-%EB%8B%A8%EC%96%B4%EB%B9%88%EB%8F%84%EC%9D%98-%EA%B0%80%EC%A4%91%EC%B9%98-TF-IDF

__리뷰별로 리스트화__ 

[리뷰1], [리뷰2],…[리뷰3] 형태의 데이터 구조

추후 topic modeling 시에 리뷰 단위로 문서를 나누고 해당 리뷰의 의미를 찾아내기 위해 리스트로 분리해서 나눠줘야 하기 때문.

In [10]:
corpus_list = list(corpus)
corpus_list[0:3]

['오류거지같음 쓸때마다 앱자체 오류도 많이 나고 이용자수 많다면서 결제도 안되고 툭하면 점검들어간다하고 동백전에 돈 충전해서 체크카드에서 출금이 됬는데 동백전에는 충전금액 뜨지도 않네요-_- 내 돈은 어디로 간거임 ㅈㄴ 별1개도 아까움',
 '국굿 아이폰 바코드나 큐알코드 결제만 앱에서 가능 하면 최고일것 같아요',
 '어플 팅김현상 2월달부터 잘 사용하다가 오늘 업데이트를 했는데 어플 접속이 안됩니다 어플 누르면 동백전 화면 뜨고 1초뒤 바탕화면으로 팅겨버려요']

#### 불용어 처리 

- 좋아요(어근 추출 시에는 '좋다'로 추출 가능하고 명사 추출 시에는 어차피 삭제되니까)는 뺐음
- 동백전,어플, 진짜, 그냥, 정말, 아주, 하니, 건가, 이면.... 등등 단어 추가했음
- korean_stopwords.csv 파일도 괜찮아보임. 지금 쓰고있는 txt 파일이 문어체스러워서? 출처는 https://blog.naver.com/yk02061/222262810714

In [11]:
def define_stopwords(path):
    
    # 방법 1: 직접 리스트에 단어 추가 
    #SW = set()
    #SW.add("동백전")
    #SW.add("앱")
    #SW.add("어플")
    
    # 방법 2: 미리 작성한 txt 파일 불러오기 
    SW = list(set())
    with open(path, encoding='cp949') as f:
        for word in f:
            SW.append(word.rstrip())

    return SW

In [12]:
SW = define_stopwords('D:/stopwords-ko.txt')

In [13]:
SW 

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

#### 사전에 특정 단어 추가

- 하나은행, 안드로이드, 캐쉬백과 같은 단어들이 쪼개지는 걸 방지하기 위함.
- 방법 1: 직접 사용자 사전 구축 
- 방법 2: customized konlpy의 add_dictionary 기능 사용 (더욱 간편, 패키지의 다른 기능들도 활용 가능)
- 참고 https://blog.naver.com/yk02061/222262810714 https://inspiringpeople.github.io/data%20analysis/ckonlpy/

In [18]:
twitter = Twitter()

twitter.add_dictionary('동백전', 'Noun')
twitter.add_dictionary('하나은행', 'Noun')
twitter.add_dictionary('부산은행', 'Noun')
twitter.add_dictionary('안드로이드', 'Noun')
twitter.add_dictionary('아이폰', 'Noun')
twitter.add_dictionary('갤럭시', 'Noun')
twitter.add_dictionary('삼성페이', 'Noun')
twitter.add_dictionary('카카오뱅크', 'Noun')
twitter.add_dictionary('뱅크샐러드', 'Noun')
twitter.add_dictionary('뱅크 샐러드', 'Noun')
twitter.add_dictionary('업데이트', 'Noun')
twitter.add_dictionary('업뎃', 'Noun')
twitter.add_dictionary('업그레이드', 'Noun')
twitter.add_dictionary('캐시백', 'Noun')
twitter.add_dictionary('캐쉬백', 'Noun')
twitter.add_dictionary('페이백', 'Noun')
twitter.add_dictionary('케이티', 'Noun')
twitter.add_dictionary('교통카드', 'Noun')
twitter.add_dictionary('10프로', 'Noun')
twitter.add_dictionary('10 프로', 'Noun')
twitter.add_dictionary('6프로', 'Noun')
twitter.add_dictionary('6 프로', 'Noun')
twitter.add_dictionary('재발급', 'Noun')
twitter.add_dictionary('지역화페', 'Noun')
twitter.add_dictionary('지역 화폐', 'Noun')
twitter.add_dictionary('고객센터', 'Noun')
twitter.add_dictionary('고객 센터', 'Noun')
twitter.add_dictionary('코나아이', 'Noun')
twitter.add_dictionary('본인인증', 'Noun')
twitter.add_dictionary('본인 인증', 'Noun')
twitter.add_dictionary('부산시', 'Noun')
twitter.add_dictionary('부산광역시', 'Noun')
twitter.add_dictionary('인터페이스', 'Noun')
twitter.add_dictionary('오프라인', 'Noun')
twitter.add_dictionary('온라인', 'Noun')
twitter.add_dictionary('회원가입', 'Noun')
twitter.add_dictionary('비번', 'Noun')
twitter.add_dictionary('비밀번호', 'Noun')
twitter.add_dictionary('아이디', 'Noun')
twitter.add_dictionary('전화번호', 'Noun')
twitter.add_dictionary('폰번호', 'Noun')
twitter.add_dictionary('생년월일', 'Noun')
twitter.add_dictionary('폰 번호', 'Noun')
twitter.add_dictionary('홈페이지', 'Noun')
twitter.add_dictionary('가맹점', 'Noun')
twitter.add_dictionary('운영사', 'Noun')
twitter.add_dictionary('소상공인', 'Noun')
twitter.add_dictionary('지원금', 'Noun')
twitter.add_dictionary('로코', 'Noun')
twitter.add_dictionary('이벤트', 'Noun')
twitter.add_dictionary('큐알', 'Noun')
twitter.add_dictionary('큐알코드', 'Noun')
twitter.add_dictionary('큐알 코드', 'Noun')
twitter.add_dictionary('큐알 코드', 'Noun')
twitter.add_dictionary('공지사항', 'Noun')
twitter.add_dictionary('공지 사항', 'Noun')
twitter.add_dictionary('불안정', 'Noun')
twitter.add_dictionary('어수선', 'Noun')
twitter.add_dictionary('번거로움', 'Noun')
twitter.add_dictionary('직관성', 'Noun')

/home/a201843240/.local/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [19]:
twitter.morphs('부산광역시 지역화폐는 동백전입니다. 운영사는 코나아이입니다')

['부산광역시', '지역화폐', '는', '동백전', '입니다', '.', '운영사', '는', '코나아이', '입니다']

__특수문자, 오타, 불필요한 자음, 숫자 등 제거__ 

- ㅋㅋㅋ와 같은 불필요한 자음
- '좋아ㅏ요'와 같은 오타 처리
- 숫자는 제거하지 않음 (10프로, 6프로 등을 지울 수 없어서) 
- 영어는 남김 (ex: UI, KT와 같은 단어를 지울 수 없어서..)

In [20]:
def message_cleaning(clean):
    
    # Series의 object를 str로 변경
    clean = [str(corpus_list) for corpus_list in clean]
    
    # 자음 모음만 있는 단어들 제거
    delete1 = re.compile("[ㄱ-ㅎ]*[ㅏ-ㅢ]*")
    clean = [delete1.sub("", corpus_list) for corpus_list in clean]
    
    # 특수문자 제거
    delete2 = re.compile("[\{\}\[\]\/?.,;:'|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]") 
    clean = [delete2.sub("", corpus_list) for corpus_list in clean]
    
    # 숫자 제거
    #delete3 = re.compile('\d+')
    #clean = [delete3.sub("", corpus_list) for corpus_list in clean]
    
    return clean

<>:11: DeprecationWarning: invalid escape sequence \{
<>:11: DeprecationWarning: invalid escape sequence \{
<>:11: DeprecationWarning: invalid escape sequence \{
<ipython-input-20-e6d91e38eff2>:11: DeprecationWarning: invalid escape sequence \{
  delete2 = re.compile("[\{\}\[\]\/?.,;:'|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]")


In [21]:
clean_list = message_cleaning(corpus)

clean_list[:3]

['업데이트도안되고 새로 지우고 설치하는데 설치가 안됩니다', '오늘 하루종일 먹통에 2중 결재 됐습니다 확인해주세요', '좋아요']

#### 필요한 품사만 추출 - 명사
(okt 트위터 형태소분석기 사용)

okt 설명 https://soyoung-new-challenge.tistory.com/31

토큰화 관련 https://wikidocs.net/21698

In [22]:
# 명사만 추출 

def noun_tokenizer(c):
    
    #okt = Okt() # 따로 다른 변수에 선언해야 함

    noun = []

    for i in range(len(c)):
        try:
            noun.append(twitter.nouns(c[i]))
        except Exception as e:
            continue
    
    return noun

In [23]:
nouns = noun_tokenizer(clean_list)

nouns[:3]

[['업데이트', '도안', '지우', '설', '치하', '데', '설치'],
 ['오늘', '하루', '종일', '먹통', '중', '결재', '확인', '해주'],
 []]

#### 불용어 및 한 글자 단어 삭제

- 한 글자 단어들 삭제
- 불용어에 속하는 단어 삭제

In [24]:
def delete_stopwords(text):
    morphs_sw = []

    for i in range(len(text)):
        t = []
        for j in range(len(text[i])):
            if len(text[i][j]) > 1 and text[i][j] not in SW: 
                t.append(text[i][j])
        morphs_sw.append(t)
       
    return morphs_sw    

In [25]:
nouns = delete_stopwords(nouns)

nouns[:3]

[['업데이트', '설치'], ['오늘', '하루', '종일', '먹통', '결재', '확인'], []]

#### 문서-단어 행렬 

In [26]:
def dic_and_bow(clean_text):
    
    # 데이터를 dictionary 형태로 명사 list 만들기 
    dictionary = corpora.Dictionary(clean_text) 
    
    # 출현빈도가 너무 적은 단어는 제거 
    dictionary.filter_extremes(no_below=5) 
    
    # 명사 형태로 말뭉치 만들기 
    corpus = [dictionary.doc2bow(text) for text in clean_text]
    
    # TF-IDF으로 변환 
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    corpus = corpus_tfidf 
    
    return corpus

참고 https://conanmoon.medium.com/%EB%8D%B0%EC%9D%B4%ED%84%B0%EA%B3%BC%ED%95%99-%EC%9C%A0%EB%A7%9D%EC%A3%BC%EC%9D%98-%EB%A7%A4%EC%9D%BC-%EA%B8%80%EC%93%B0%EA%B8%B0-58-8f05f611d783

In [27]:
corpus = dic_and_bow(nouns)

corpus[0][:3]

[(0, 0.7822808035725777), (1, 0.6229259541565452)]

#### 최적 Topic 개수 산출

<span style="color:red">coherence 보단 perplexity를 우선적으로 보는 게 좋음</span>


__(1) Coherence Model__

Topic이 얼마나 의미론적으로 일관성 있는지 판단.
수치가 높을수록 일관성 높음. 0.55 정도면 우수하다고 함.
Coherence가 너무 높아지면 정보의 양이 줄어들게 되고, coherence가 너무 낮아 정보들이 인관성이 없다면 분석의 의미가 낮아지게 됨.

.3 is bad

.4 is low

.55 is okay

.65 might be as good as it is going to get

.7 is nice

.8 is unlikely and

.9 is probably wrong

__(2) 언어 모델 평가방법 (PPL: Perplexity)__

확률 모델이 결과를 얼마나 정확하게 예측하는지 나타내는 값.
동일 모델 내 파라미터에 따른 성능 평가할 때 사용.
선정된 토픽 개수마다 학습시켜 가장 낮은 값을 보이는 구간을 찾아 최적화된 토픽의 개수 선정. 
수치가 낮을수록 높은 정확도.
Coherence Score와 함께 고려해야함.

__LDA 시각화__ 

adjust your parameters alpha = .1, beta = .01 or .001, random_state = 123

In [28]:
topics = 8

In [29]:
dictionary = corpora.Dictionary(nouns)   
dictionary.filter_extremes(no_below=5) 

In [30]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=topics, 
                                           alpha="auto", 
                                           eta="auto",
                                           #gamma_threshold=0.9,
                                           id2word=dictionary,
                                           #chunksize=100,
                                           passes=10,
                                           iterations=1000,
                                           random_state=100) 

http://bigdata.emforce.co.kr/index.php/2020072401/ 파라미터 설명 

파이썬에서 alpha 값은 1/k(토픽개수)

beta(β)는 0.01로 지정한다(Lin and He, 2009)

λ = 1 일수록 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미, 

λ = 0 일수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미(해당 토픽에서 많이 등장한 단어)

In [31]:
ldamodel.print_topics(num_words=10)

[(0,
  '0.247*"사용" + 0.040*"필요" + 0.039*"경제" + 0.037*"부산" + 0.034*"오류" + 0.029*"지역" + 0.021*"유용" + 0.019*"카드" + 0.019*"화이팅" + 0.019*"충전"'),
 (1,
  '0.068*"가맹점" + 0.043*"적립" + 0.035*"찾기" + 0.032*"짜증" + 0.031*"설치" + 0.024*"다시" + 0.024*"그냥" + 0.023*"삭제" + 0.023*"계속" + 0.021*"사용"'),
 (2,
  '0.138*"접속" + 0.060*"네트워크" + 0.037*"캐쉬백" + 0.033*"에러" + 0.026*"상태" + 0.025*"실행" + 0.024*"연결" + 0.023*"원활" + 0.019*"적립금" + 0.019*"오늘"'),
 (3,
  '0.060*"카드" + 0.054*"부산" + 0.039*"유용" + 0.032*"신청" + 0.027*"사용" + 0.026*"가능" + 0.019*"사람" + 0.019*"지역화폐" + 0.018*"문자" + 0.018*"와이파이"'),
 (4,
  '0.071*"결제" + 0.043*"금액" + 0.041*"충전" + 0.034*"인증" + 0.029*"번호" + 0.024*"카드" + 0.022*"오류" + 0.021*"확인" + 0.021*"다운" + 0.020*"불가"'),
 (5,
  '0.159*"업데이트" + 0.103*"캐시백" + 0.082*"혜택" + 0.055*"불편" + 0.053*"사용" + 0.033*"자주" + 0.031*"아직" + 0.030*"도움" + 0.023*"지금" + 0.022*"로그인"'),
 (6,
  '0.123*"편리" + 0.121*"최고" + 0.077*"포인트" + 0.061*"실행" + 0.040*"10프로" + 0.032*"계좌" + 0.028*"적립" + 0.025*"사용" + 0.023*"잔액" + 0.019*"아이폰"'),
 (7,
  '0

In [32]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.054842 -0.166440       1        1  15.715548
4     -0.055265 -0.056661       2        1  14.464937
1     -0.056767  0.017711       3        1  13.601242
2     -0.157905  0.070398       4        1  12.914766
5      0.262457  0.098281       5        1  11.838371
6      0.013821  0.117422       6        1  11.438734
7     -0.082020  0.080082       7        1  10.068225
0      0.130521 -0.160793       8        1   9.958177, topic_info=      Term        Freq       Total Category  logprob  loglift
40      사용  102.000000  102.000000  Default  30.0000  30.0000
1     업데이트   51.000000   51.000000  Default  29.0000  29.0000
53      접속   49.000000   49.000000  Default  28.0000  28.0000
76      최고   32.000000   32.000000  Default  27.0000  27.0000
110     편리   35.000000   35.000000  Default  26.0000  26.0000
102     감사   25.000000   25.000000  Default  25.0000  25.0000
49     캐시백   35.000000   35.000000  Default  24.0000  24.0000
128     혜택   25.000000   25.000000  Default  23.0000  23.0000
54     포인트   21.000000   21.000000  Default  22.0000  22.0000
21      만족   16.000000   16.000000  Default  21.0000  21.0000
41     가맹점   25.000000   25.000000  Default  20.0000  20.0000
83      부산   29.000000   29.000000  Default  19.0000  19.0000
44      결제   32.000000   32.000000  Default  18.0000  18.0000
9       계속   31.000000   31.000000  Default  17.0000  17.0000
184     할인   13.000000   13.000000  Default  16.0000  16.0000
28    네트워크   27.000000   27.000000  Default  15.0000  15.0000
29      실행   33.000000   33.000000  Default  14.0000  14.0000
37      유용   19.000000   19.000000  Default  13.0000  13.0000
75      적립   24.000000   24.000000  Default  12.0000  12.0000
10      금액   16.000000   16.000000  Default  11.0000  11.0000
74    10프로   13.000000   13.000000  Default  10.0000  10.0000
125     필요   10.000000   10.000000  Default   9.0000   9.0000
93      불편   27.000000   27.000000  Default   8.0000   8.0000
111     경제    9.000000    9.000000  Default   7.0000   7.0000
42      찾기   11.000000   11.000000  Default   6.0000   6.0000
25      짜증   10.000000   10.000000  Default   5.0000   5.0000
133     신청   14.000000   14.000000  Default   4.0000   4.0000
32      계좌   13.000000   13.000000  Default   3.0000   3.0000
70      에러   11.000000   11.000000  Default   2.0000   2.0000
109    캐쉬백   18.000000   18.000000  Default   1.0000   1.0000
91      택시    5.921725    6.737208   Topic1  -4.1095   1.7215
166   지역화폐    6.725030    8.078924   Topic1  -3.9823   1.6671
213     간단    5.534659    6.723704   Topic1  -4.1771   1.6559
215     시민    3.592796    4.400079   Topic1  -4.6092   1.6478
133     신청   11.625477   14.431654   Topic1  -3.4349   1.6343
145    드디어    3.851099    4.848655   Topic1  -4.5398   1.6202
131     문자    6.608653    8.763318   Topic1  -3.9997   1.5683
12      답변    5.009229    6.712217   Topic1  -4.2768   1.5579
127     알림    4.008738    5.462751   Topic1  -4.4996   1.5410
37      유용   14.072589   19.593152   Topic1  -3.2439   1.5196
153     본인    3.285550    4.607563   Topic1  -4.6986   1.5124
67      수정    3.725933    5.592634   Topic1  -4.5728   1.4444
83      부산   19.563135   29.987103   Topic1  -2.9145   1.4234
98      마음    2.200886    3.485238   Topic1  -5.0993   1.3908
204   하나은행    1.995470    3.255024   Topic1  -5.1972   1.3612
216     리뷰    2.296753    3.791278   Topic1  -5.0566   1.3493
84      사람    6.874319   11.423844   Topic1  -3.9603   1.3426
194     기존    1.554025    2.589589   Topic1  -5.4473   1.3399
68      이유    2.240693    3.762289   Topic1  -5.0813   1.3323
198     발전    2.657464    4.488238   Topic1  -4.9107   1.3264
108   체크카드    3.228080    5.491066   Topic1  -4.7162   1.3193
17      전화    5.445255    9.287858   Topic1  -4.1934   1.3166
170     제도    2.563817    4.465358   Topic1  -4.9466   1.2957
107     처음    3.970510    7.057850   Topic1  -4.5092   1.2753
180    

In [33]:
word_dict = {};

for i in range(topics):

    words = ldamodel.show_topic(i)

    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]

pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,사용,가맹점,접속,카드,결제,업데이트,편리,감사
1,필요,적립,네트워크,부산,금액,캐시백,최고,계속
2,경제,찾기,캐쉬백,유용,충전,혜택,포인트,만족
3,부산,짜증,에러,신청,인증,불편,실행,할인
4,오류,설치,상태,사용,번호,사용,10프로,오류
5,지역,다시,실행,가능,카드,자주,계좌,하루
6,유용,그냥,연결,사람,오류,아직,적립,네트워크
7,카드,삭제,원활,지역화폐,확인,도움,사용,종일
8,화이팅,계속,적립금,문자,다운,지금,잔액,연결
9,충전,사용,오늘,와이파이,불가,로그인,아이폰,기대


참고: 아마존 고객리뷰 토픽모델링 https://blog.naver.com/upennsolution/221437143732

#### 토픽별 문서

In [131]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

참고한 코드 출처 https://wikidocs.net/30708

사용한 코드는 아니나 혹시나 싶어 https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#15visualizethetopicskeywords

In [132]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 리뷰번호를 의미하는 열로 사용하기 위해서 인덱스 열을 하나 더 만들어줌
topictable.columns = ['리뷰 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,리뷰 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,1.0,0.4132,"[(0, 0.04898984), (1, 0.41317058), (2, 0.04936..."
1,1,7.0,0.7483,"[(0, 0.034349125), (1, 0.03494677), (2, 0.0346..."
2,2,3.0,0.1353,"[(0, 0.120294034), (1, 0.1218614), (2, 0.12116..."
3,3,3.0,0.5750,"[(0, 0.059091624), (1, 0.059859972), (2, 0.059..."
4,4,3.0,0.4009,"[(0, 0.026261106), (1, 0.026635647), (2, 0.026..."
5,5,7.0,0.5673,"[(0, 0.059089605), (1, 0.059859503), (2, 0.059..."
6,6,1.0,0.7312,"[(0, 0.03679822), (1, 0.73121375), (2, 0.03706..."
7,7,3.0,0.1353,"[(0, 0.120294034), (1, 0.1218614), (2, 0.12116..."
8,8,3.0,0.1353,"[(0, 0.120294034), (1, 0.1218614), (2, 0.12116..."
9,9,3.0,0.1353,"[(0, 0.120294034), (1, 0.1218614), (2, 0.12116..."


##### 토픽 1

'0.247*"사용" + 0.040*"필요" + 0.039*"경제" + 0.037*"부산" + 0.034*"오류" + 0.029*"지역" + 0.021*"유용" + 0.019*"카드" + 0.019*"화이팅" + 0.019*"충전"'

In [133]:
topic_1 = topictable[topictable['가장 비중이 높은 토픽'] == 0.0]

print('토픽 1에 해당하는 리뷰의 개수:', topic_1.shape[0], '개')

토픽 1에 해당하는 리뷰의 개수: 137 개


In [134]:
# 토픽 1에 해당하는 리뷰들만 모아서 하나의 df 만들기

topic_1_list = topic_1['리뷰 번호'].unique()
df_topic_1 = df_before.loc[topic_1_list, :]

df_topic_1.head(5)

,DATE,STAR,REVIEW,LIKE
21,2020-01-06,1,개인 정보 입력만 13회 했습니다. 이 어이없는 오류 뭡니까?,1
30,2020-04-13,4,사용하기좋아ㅡ여,0
31,2020-04-29,5,소상공인도 부산시민도 다 유용한카드~~,0
36,2020-01-07,1,앱 사용 내역 볼려고하면 계에에에에ㅔㅔㅔㅔㅔ속 나가지네요 빨리 고쳐 주세요,0
39,2020-04-13,5,"사용하기 편하고, 적립도 많고, 지역 상인도 도움되고.좋아요",0


In [135]:
# 리뷰 전부 모아보기 --> 복사해서 R로 들고가서 감성분석 진행

df_topic_1['REVIEW'].unique()

array(['개인 정보 입력만 13회 했습니다. 이 어이없는 오류 뭡니까?', '사용하기좋아ㅡ여',
       '소상공인도 부산시민도 다 유용한카드~~',
       '앱 사용 내역 볼려고하면 계에에에에ㅔㅔㅔㅔㅔ속 나가지네요 빨리 고쳐 주세요 ',
       '사용하기 편하고, 적립도 많고, 지역 상인도 도움되고.좋아요',
       '편리하게 사용할수 있어좋은것같고캐시가쌓여서좋은것같네요',
       '동백전으로 여기저기 많이 사용해 볼게요. 부산 경제에도 많은 도움이 되면 좋겠습니다.',
       '사용이 편리하고 앱 디자인도 멋집니다.', '부산거주자 강추',
       '카드로.. 삼성페이로.. 사용하기 매우 간편하고, 바로바로 적립되는 캐시백 보면서 소비가 더 즐거워집니다. 좋아요~^^',
       '사용이.편합니다', '굿♥ 두말할 필요없습니다', '월별 사용합계를 자동으로 볼수 있으면 더 좋을듯해요',
       '경제 살림에 정말 굿 행복한 날을 위해ᆢ 감사합니다 ㅎ',
       '부산경제를 위한 동백전이네요.. 10% 혜택을 주니 잘 사용하겠습니다. 앱 사용이 편리하네요.',
       '사용 편하고 좋아요', '지역경제도 살리고 우리집에도 도움되는 최고 앱', '너무 강추', '사용하기 쉬움',
       '부산을 대표하는 꽃 이름으로 지은 동백전. 이름이 멋지네요. 부산에서 이용하는 혜택도 많아 기대가 됩니다. 앱도 복잡하지 않고 사용하기 편리해 보여 한번 써 봐야겠네요.',
       '경제절약으로 좋아요',
       '괜찮게 잘 쓰고 있어요. 그런데 앱이 자주 다운 되고 안 열릴때가 종종 발생해요 당장 필요할때 앱이 안열리니 살짝 멘붕이...',
       '제안을 하고자 합니다. 어린이들도 동백전 사용자로 참여하도록 하면 어떨까요? 인천의 인천e음카드처럼 6~13세 부산 어린이를 위한 어린이용 동백전이 발급되면 좋겠어요. 체크카드 기능을 빼고 선불 충전방식에 부산을 상징하는 캐릭터가 들어간 디자인의 

##### 토픽 2

'0.068*"가맹점" + 0.043*"적립" + 0.035*"찾기" + 0.032*"짜증" + 0.031*"설치" + 0.024*"다시" + 0.024*"그냥" + 0.023*"삭제" + 0.023*"계속" + 0.021*"사용

In [136]:
topic_2 = topictable[topictable['가장 비중이 높은 토픽'] == 1.0]

print('토픽 2에 해당하는 리뷰의 개수:', topic_2.shape[0], '개')

토픽 2에 해당하는 리뷰의 개수: 167 개


In [137]:
# 토픽 2에 해당하는 리뷰들만 모아서 하나의 df 만들기

topic_2_list = topic_2['리뷰 번호'].unique()
df_topic_2 = df_before.loc[topic_2_list, :]

df_topic_2.head(5)

,DATE,STAR,REVIEW,LIKE
0,2020-05-13,1,업데이트도안되고 새로 지우고 설치하는데 설치가 안됩니다.,0
6,2020-03-10,1,가입이 안됩니다!!인증이 안되요~ㅎㅎ발급 받아보려고 어플깔고 휴대폰 인증하는거 아무...,1
16,2020-02-09,3,가맹점 찾기를 통해 나오는 가맹점이 10년전쯤에 그 자리에 있던걸로 현재는 없어졌는...,0
27,2020-04-13,4,월에 얼마썻는ㄷ지 안떠서 별로네요 진짜 돈많이쓰게 노린듯,0
32,2020-04-19,1,"어플 실행때마다, 인증된 폰번호와 현재 폰번호가 일치하지 않습니다... 글귀가 나와...",2


In [138]:
# 리뷰 전부 모아보기 --> 복사해서 R로 들고가서 감성분석 진행

df_topic_2['REVIEW'].unique()

array(['업데이트도안되고 새로 지우고 설치하는데 설치가 안됩니다.',
       '가입이 안됩니다!!인증이 안되요~ㅎㅎ발급 받아보려고 어플깔고 휴대폰 인증하는거 아무리 눌러도 안되니 신경질나서 쓰겠나싶어요~나만 안되는거야!!!??짜증나요^^',
       '가맹점 찾기를 통해 나오는 가맹점이 10년전쯤에 그 자리에 있던걸로 현재는 없어졌는데도 나오네요;;;',
       '월에 얼마썻는ㄷ지 안떠서 별로네요 진짜 돈많이쓰게 노린듯',
       '어플 실행때마다, 인증된 폰번호와 현재 폰번호가 일치하지 않습니다... 글귀가 나와 재인증해 사용하는데요. 알뜬폰이라 그런건지, 이번에 업데이트해도 동일하네요. 폰번호가 맞는데 이유를 모르겠는요, 담에 수정 부탁드립니다.',
       '이게 뭔가요? 계속 열리지도 않고 짜증 너무 나네요',
       '진짜 허접한 앱때매 사용하기 어렵네요.. 도대체 뭐하자는 겁니까?',
       'ㅠ가맹점 찾기라뇨ㅠ 생각보다 안되는곳도 많고 충전금액 환불은 왜 안되는거죠? 계속 실패했다고 뜨는데',
       '앱을 열때 마다 폰번호와 맞지않다고 나오고 초기화하라고 해서 몇번을 다시 했는지 모릅니다. 돈 입금시켜 놓고 사용할려고 하는데 이따위로 앱을 만들어 나서 너무 불편합니다.',
       '대형마트와백화점등대형업체만빼면 대부분사용되네요 가맹점찾기어렵다는분이해가안되는데 동네병원도되고 약국도되고 슈퍼,빵집,음식점,주유소...안되는곳찾기가더힘듦 도대체어디사시는분인지궁금하네요 캐시백도결재와동시에딱들어오고 첨엔되나안되나가맹점찾기도해봤는데 이젠필요없어요 신용카드쓸데가없어지고있어서그게문제 쓰는재미가쏠쏠합니다~',
       '카드신청시 일정시간미사용으로 거래불가가 계속 뜨는데 제거후 다시 설치해도 그렇네요 어떻게 해결해야 할까요?',
       '아니 접속이 자꾸 안되는데 어째된거죠 짜증나네요 이젠',
       '그냥 업데이트중이라고 뜨면서 작동할생각을안함 여기돈넣고쓴 내가호구인듯', '그냥좋아요',
     

##### 토픽 3

0.138*"접속" + 0.060*"네트워크" + 0.037*"캐쉬백" + 0.033*"에러" + 0.026*"상태" + 0.025*"실행" + 0.024*"연결" + 0.023*"원활" + 0.019*"적립금" + 0.019*"오늘

In [139]:
topic_3 = topictable[topictable['가장 비중이 높은 토픽'] == 2.0]

print('토픽 3에 해당하는 리뷰의 개수:', topic_3.shape[0], '개')

토픽 3에 해당하는 리뷰의 개수: 166 개


In [140]:
# 토픽 3에 해당하는 리뷰들만 모아서 하나의 df 만들기

topic_3_list = topic_3['리뷰 번호'].unique()
df_topic_3 = df_before.loc[topic_3_list, :]

df_topic_3.head(5)

,DATE,STAR,REVIEW,LIKE
10,2020-04-14,1,앱이 실행이안됨. 네트워크 잘만 연결되어있는데도 원활하지않다고 빠꾸먹임 관리좀잘해라...,1
22,2020-03-16,2,네트워크 오류라면서 어플 접속이 안될뿐더러 오늘 실 결제한 금액 캐시백도 안됐네요....,1
33,2020-04-13,1,에러가많이나요,0
40,2020-01-09,5,캐시백 개꿀~!,0
50,2020-04-10,1,앱관리 안하냐 ? 네트워크 접속불량 머냐,0


In [141]:
# 리뷰 전부 모아보기 --> 복사해서 R로 들고가서 감성분석 진행

df_topic_3['REVIEW'].unique()

array(['앱이 실행이안됨. 네트워크 잘만 연결되어있는데도 원활하지않다고 빠꾸먹임 관리좀잘해라 진짜ㅡㅡ 고쳐줘 빨리',
       '네트워크 오류라면서 어플 접속이 안될뿐더러 오늘 실 결제한 금액 캐시백도 안됐네요...뭐지요?', '에러가많이나요',
       '캐시백 개꿀~!', '앱관리 안하냐 ? 네트워크 접속불량 머냐', '에러작렬',
       '폰마다 차이가있는지 제폰에서는 와이파이 아니면 앱에 접속이 거의 안됩니다(A5). 길거리에서 갑자기 충전해야할때 또는 가맹점확인해야할때 LTE면 네트워크 상태가 원활하지 않다며 바로 꺼집니다. 확인이 불가능해요. 길 한복판에서 와이파이 잡는게 쉬운것도 아닌데ㅜ 너무너무너무 불편하고 급할때는 정말 욕나와요ㅠㅠㅠ 어제도 앱 50번 출구컷 당하고 물건 못사고 나왔습니다. 개선 안됩니까',
       '네트워크 네트워크로 인해 실행이 안된다고 뜨네욤,,, 아까 까지 되었는데,,, 수정해주세욤', '캐시백 개꿀',
       '적립한도 볼 수 있게 잘 쓰고 있는데 충전한도 말고 적립한도도 볼 수 있게 하면 훨씬 좋을 것 같네요~~',
       '자꾸 네트워크 이상이라면서 안 들어가집니다 급하게 써야할때마다 난감해요 폰 기종이 갤럭시 노트4라서 그런건가요?',
       '네트워크 연결이 안된다고 합니다', '저는잘되는데갤럭시s8입니다 안된다는분안타깝네요 ㅜ',
       '동백전카드 가입에서 비밀번호 실패하고 나서는 접속하기가 힘들어요~~~ 충전시킬려고 하는데 본인인증이 안 됩니다ㅠㅠ',
       '개꿀~', '캐쉬백 좋네요', '앱 접속 좀 하자. 지금 몇일째 접속이 안되는건지. 관리할 마음은 있는건지.',
       '앱은 설치했으나 ᆞ 네트워크 원활하지 않다고 압접속않됩니다', '개느리고 접속 안됩니다.',
       '접속이안돼요 충전은어찌해야하나',
       '동백전 최고 동백전 충전하면 캐쉬백도 10%나 해주고 가정생활에 많은 도움이 되네요 지하철도 되고, 또 계좌연결하면 카드처

##### 토픽 4

0.060*"카드" + 0.054*"부산" + 0.039*"유용" + 0.032*"신청" + 0.027*"사용" + 0.026*"가능" + 0.019*"사람" + 0.019*"지역화폐" + 0.018*"문자" + 0.018*"와이파이"

In [142]:
topic_4 = topictable[topictable['가장 비중이 높은 토픽'] == 3.0]

print('토픽 4에 해당하는 리뷰의 개수:', topic_4.shape[0], '개')

토픽 4에 해당하는 리뷰의 개수: 751 개


In [143]:
# 토픽 4에 해당하는 리뷰들만 모아서 하나의 df 만들기

topic_4_list = topic_4['리뷰 번호'].unique()
df_topic_4 = df_before.loc[topic_4_list, :]

df_topic_4.head(5)

,DATE,STAR,REVIEW,LIKE
2,2020-04-30,5,좋아요,0
3,2020-04-10,1,와파든 데이터든 안들어가져요,0
4,2020-04-16,1,카드취소후 동백전금액이 회복이 안되어 문의할래도 하루종일 전화연결은 안되고 지난번 ...,1
7,2020-03-03,5,좋아요,0
8,2020-09-08,5,펀리해요,0


In [144]:
# 리뷰 전부 모아보기 --> 복사해서 R로 들고가서 감성분석 진행

df_topic_4['REVIEW'].unique()

array(['좋아요', '와파든 데이터든 안들어가져요',
       '카드취소후 동백전금액이 회복이 안되어 문의할래도 하루종일 전화연결은 안되고 지난번 1대1문의글남긴거 1달지나도 답변대기중 답도없고 오늘 통화연결계속안되어 포기하고 문의란 다시 글남겼는데 언제답변올지 도대체 이래가 쓰겠나요?',
       '펀리해요', '채 성진', '계좌등록만하면 간편하게 충전, 확인.가능해요. 다 좋은데 카카오뱅크도 추가해주세요ㅠ',
       '간편하게 유용하게 잘 활용할 수 있어요', '간편하게 잘 만들어진 앱 입니다. 사용방법도 쉽군요.', '조아요',
       '너~쪼아용', '좋은어플입니다', '굿',
       '교통카드 없는 카드 개설했다가 있는걸로 바꿀려고 하나카드 통해서 카드해지를 했습니다. 근데 어플상에 정보가 그대로 있어서 재발급을 하지못하고 있습니다. 고객센터 또한 하루 30통 이상씩하는데도 연결불가 영상통화비용으로 11000원이 부과되었네요? 상담조차 못했는데 말이죠? 아주 답답한 경우라 별 1개도 아깝네요 1대1 문의도 답이없고 여기에 달릴 답변은 고객센터에 전화해봐라 이 얘기겠죠?',
       '잘쓰고있어요', 'Good job', '신통방통합니다', '실물카드받았는데 어플엔 등록안되어있어요',
       '슈퍼에서 우유를 2 번 샀읍니다 .10%씩 적립 되여 기분 좋았읍니다. 택시는 개인 도 일반 택시도 안되더라고요. 언제쯤 택시 될가요.?!',
       'ㄷㄱ디', '택시는 사용 안되나요?', '편하게 잘쓰고 있습니다', '부산은행은 결제계좌로등록이안됨',
       '너무 좋았어요', '군더더기 없는 ui가 마음에 드는군요. 많은 사업장에서 사용가능하면 더 좋겠습니다.',
       '좋아요..',
       '동백점 미가맹점 결제 가능 요청 잘 사용중입니다. 감사합니다^^* 다만 동백점 미가맹점에도 일반 체크카드처럼 캐쉬백은 안되어도 결제는 가능했으면 좋겠습니다. 처음 방문하는 사업장에 카드를 주면 결제 실패로 

##### 토픽 5

'0.071*"결제" + 0.043*"금액" + 0.041*"충전" + 0.034*"인증" + 0.029*"번호" + 0.024*"카드" + 0.022*"오류" + 0.021*"확인" + 0.021*"다운" + 0.020*"불가

In [145]:
topic_5 = topictable[topictable['가장 비중이 높은 토픽'] == 4.0]

print('토픽 5에 해당하는 리뷰의 개수:', topic_5.shape[0], '개')

토픽 5에 해당하는 리뷰의 개수: 169 개


In [146]:
# 토픽 5에 해당하는 리뷰들만 모아서 하나의 df 만들기

topic_5_list = topic_5['리뷰 번호'].unique()
df_topic_5 = df_before.loc[topic_5_list, :]

df_topic_5.head(5)

,DATE,STAR,REVIEW,LIKE
17,2020-01-07,1,지웠다 깔아도 들어갈 수가 없음 -_- 마트에서는 결제가 안됐다고 하고 돈이 빠졌는...,0
26,2020-04-13,4,좋긴좋아요. 포인트 는 서버가자꾸 다운되요,0
34,2020-03-11,4,"잘 사용하고 있습니다. 아쉬운건 잔액정보인데, 1.사용 전 금액 상품권 100원 캐...",10
41,2020-01-07,4,교통카드 이용내역 다 좋은데 카드 이용 내역에 후불교통카드로 쓰인 금액도 적혀있으면...,0
62,2020-04-29,1,서버 다운인 상황에 결제가 안될때 난감한경우가 제법 있습니다.,0


In [147]:
# 리뷰 전부 모아보기 --> 복사해서 R로 들고가서 감성분석 진행

df_topic_5['REVIEW'].unique()

array(['지웠다 깔아도 들어갈 수가 없음 -_- 마트에서는 결제가 안됐다고 하고 돈이 빠졌는지 확인을 해야 하는데 이 무슨 xx같은 상황인지,,, 쓸때마다 왜 이런일이 일어나는지.. 충전하고나서 어플에 안들어가짐 네트워크 상태 확인하라면서 별하나도 아깝',
       '좋긴좋아요. 포인트 는 서버가자꾸 다운되요',
       '잘 사용하고 있습니다. 아쉬운건 잔액정보인데, 1.사용 전 금액 상품권 100원 캐시뱁 10원 합계 110 원 2.사용 금액 상품권 90원 캐시백 10원 합계 100원 3.잔액 상품권 10 원 캐시백 0원 합계 10 원 이런식으로 사용 전, 후 정보를 제공해주셨으면 합니다',
       '교통카드 이용내역 다 좋은데 카드 이용 내역에 후불교통카드로 쓰인 금액도 적혀있으면 좋을 것 같아요',
       '서버 다운인 상황에 결제가 안될때 난감한경우가 제법 있습니다.',
       '동백전은 좋은데 어플이 왜이래요? 결제가 두번 된거 같아 확인하려고 하니 네트워크 불량이라고 연결이 안되요 또해도 안되요 계속해도 안되요 나중에 집에가서 결제 두번된거 확인하고 다시 돌아가서 결제 취소하고 돌아왔어요 제발 쫌 어플 관리 좀 잘해주세요',
       '설치하고 카드 신청하는데 폰으로 인증번호 확인차 문자앱 실행해서 보고 다시 동백전앱에 인증번호 넣을려고 클릭하면 재시작 되어서 다시 휴대폰 번호 넣어라함ㅡ.ㅡ 몇번째 다시하다가 성질나서 카드신청 안함ㅡ 먼 앱을 이따위로 만들었는지ㅡ.ㅡ',
       '어플이 진짜 구려요 오류나서 접속이 안되고 로그인도 안되고..써야할 때 못쓰면 무슨 소용이죠 시스템안정화 신경써주세요',
       '결제오류가 타카드에 비해 너무 자주나요', '편리하고 좋아요, 다만 잔액부족시 바로 카드결제 되는점은 개선 바랍니다',
       '실행이안됩니다 개선해주세요',
       '2g폰 사용하고 있는데 동백전 발급받을 수 있는 방법은 없나요? 예를들면 아이패드나 갤럭시탭 또는 공폰 이용하여 앱 다운받은

##### 토픽 6

0.159*"업데이트" + 0.103*"캐시백" + 0.082*"혜택" + 0.055*"불편" + 0.053*"사용" + 0.033*"자주" + 0.031*"아직" + 0.030*"도움" + 0.023*"지금" + 0.022*"로그인

In [148]:
topic_6 = topictable[topictable['가장 비중이 높은 토픽'] == 5.0]

print('토픽 6에 해당하는 리뷰의 개수:', topic_6.shape[0], '개')

토픽 6에 해당하는 리뷰의 개수: 174 개


In [149]:
# 토픽 6에 해당하는 리뷰들만 모아서 하나의 df 만들기

topic_6_list = topic_6['리뷰 번호'].unique()
df_topic_6 = df_before.loc[topic_6_list, :]

df_topic_6.head(5)

,DATE,STAR,REVIEW,LIKE
20,2020-04-23,1,캐시백 재원부족이라더니...업데이트를 해야 충전 할 수 있는데 수일째 업데이트가 안...,0
37,2020-04-28,5,바로 바로 캐시백~,0
46,2020-04-10,5,지금 사용이 안되네요.,1
48,2020-04-29,4,폰으로 충전이 안되서 불편하네요 어떻게 할수 있는지~~?,0
58,2020-04-29,5,"10%캐시백 큰 액수죠,,",0


In [150]:
# 리뷰 전부 모아보기 --> 복사해서 R로 들고가서 감성분석 진행

df_topic_6['REVIEW'].unique()

array(['캐시백 재원부족이라더니...업데이트를 해야 충전 할 수 있는데 수일째 업데이트가 안됨 결국 사용하지 마라는건지..',
       '바로 바로 캐시백~', '지금 사용이 안되네요.', '폰으로 충전이 안되서 불편하네요 어떻게 할수 있는지~~?',
       '10%캐시백 큰 액수죠,,', '안드로이드11로 업데이트 후 알림이 부분적으로 오지 않습니다.',
       '잘 사용해 볼게요. 동백전으로 혜택 많이 받으면 좋겠습니다.',
       '업데이트좀 .. 급하게 사용해야하는데 업데이트하게 만드네요', '캐시백이 너무 좋고 재미있어요~~^^',
       '앱이 열려야 충전해서 사용을 하죠 안열리는 경우가 너무 많아요 네트워크가 원활하지않다고 자주떠요',
       '업데이트가너무힘드네요', '아이폰 업데이트된 이후에 안되는건지 동백전이 저번주까진 열렸는데 이번주에 내내 안열립니다',
       '불편없이 사용 잘 합니다', '캐시백 환원요',
       '잘 사용하고 있는데 괜한 업데이트를 해야만 사용할수 있게해서 지금현재 마음대로 사용할수없어서 넘 불만입니다 업데이트가 안되네요 2일이나 시도했는데 대기중만 나오고 안되서 동백전 사용도 못하고 있어요',
       '좋긴한데요…. 결제 어플인데 이거 업뎃때문에 못쓴다고 안 막았으면 합니다. 결제해야해서 잔액 보려고 들어가니 업데이트 하라고 어플이 안 켜지다니요 ㅠ 업데이트 공지 올려주시고 일단 주요 기능은 쓸 수 있게 해주세요.',
       '앱 업데이트를 해야 이용가능하다는데 업데이트가 안되네요', '앱이 오류가 자주 뜨고 메뉴얼이 불편하게 되어 있어요.',
       '사용중 오류로...돈 손해봄...ㅠ 그래도 이만한 것은 아직 없으니....', '캐시백이 좋아요',
       '혜택을 많이 준다해서 하나장만했더니 무슨 행정을 손바닥 뒤집듯바꾸나요? 재수없어서 짤라 버려야겠다',
       '로그인이 안되네요', '캐시백 넘좋아요', '동백전 넘조아용 지금까지 내가 

##### 토픽 7

'0.123*"편리" + 0.121*"최고" + 0.077*"포인트" + 0.061*"실행" + 0.040*"10프로" + 0.032*"계좌" + 0.028*"적립" + 0.025*"사용" + 0.023*"잔액" + 0.019*"아이폰

In [151]:
topic_7 = topictable[topictable['가장 비중이 높은 토픽'] == 6.0]

print('토픽 7에 해당하는 리뷰의 개수:', topic_7.shape[0], '개')

토픽 7에 해당하는 리뷰의 개수: 164 개


In [152]:
# 토픽 7에 해당하는 리뷰들만 모아서 하나의 df 만들기

topic_7_list = topic_7['리뷰 번호'].unique()
df_topic_7 = df_before.loc[topic_7_list, :]

df_topic_7.head(5)

,DATE,STAR,REVIEW,LIKE
23,2020-04-28,5,포인트 쓰는맛이 쏠쏠하네요^^👍👍👍,0
35,2020-04-28,5,적립 10프로 최고예요~,0
60,2020-02-28,4,다 좋은데 연결계좌 은행에 카카오뱅크가 없어서 불편합니다. 연결계좌은행은 좀 다양해...,2
77,2020-11-07,5,편리함,0
79,2020-09-13,3,한달총액이 한눈에 보였으면 합니다 한달 총액으로 캐시백포인트비율이 달라지는 걸 감안...,0


In [153]:
# 리뷰 전부 모아보기 --> 복사해서 R로 들고가서 감성분석 진행

df_topic_7['REVIEW'].unique()

array(['포인트 쓰는맛이 쏠쏠하네요^^👍👍👍', '적립 10프로 최고예요~',
       '다 좋은데 연결계좌 은행에 카카오뱅크가 없어서 불편합니다. 연결계좌은행은 좀 다양해져도 좋을것 같아서 별하나 뺐습니다.',
       '편리함', '한달총액이 한눈에 보였으면 합니다 한달 총액으로 캐시백포인트비율이 달라지는 걸 감안해 주세요', '최고',
       '업데이트3일동안 안되고 앱실행이안되여', '어휴 어플관리 진짜 최악이다', '포인트도쓰고 넘좋네요',
       '카카오뱅크는 왜 등록이 안되요? 카카오뱅크는 왜 등록이 안되요?', '서버 좀ㅡㅡ',
       '앱 실행이 ㅡㅡ 안되요 첫결제하고 포인트확인하려는데 종일 안됩니다 머 이런 어플이 있나 부산사람들 성격 급한거 모르나 ㅡㅡ',
       '머고 앱실행도안되고 장난하냐???!!!', '포인트 적립되니 너무좋아요 ㅎ', '이득이 되고 편리함',
       '오늘카드를 만들어서 사용했는데 적립10프로쌓이는것도 바로 확인되서 좋아요...',
       '최고에요. 계속되는 10% 적립혜택도 좋고, UI도 신경써서 만든 것 같아요. 편리합니다.',
       '어플보다..동백전10프로적립이너무좋네요', '무조건 좋습니다~~^^',
       '아니... 계좌등록하려니 계속 계좌번호 오류라고하는데 왜그런거예요? 숫자 잘못친줄알고 10번은 더 넣어좟어여!!!!! 심지어 2개 계좌 다 안되는데... 하... 화가나네요.',
       '앱이 안열림 ㅡ.ㅡ뭐하자는거임? 사용자 증가로 오류? 골때리네',
       '소상공인도 돕고 사용자도 포인트 받고.. 아주 좋습니다. 연장해주시길..',
       '카드를 받고 앱을 실행할려니 안되네요. 이렇게 허접하다니?',
       '가맹점이 어디더라 생각 안하고 일단 들어가서 결재하면 어지간하면 가맹점이네요 ㅋㅋ 10프로 캐시백 이벤트 오래오래 했음 좋겠어요! 3월까지!',
       '포인트적립 넘 맘에들어요', '좋아요 편리하고 ^^

##### 토픽 8

0.100*"감사" + 0.073*"계속" + 0.069*"만족" + 0.056*"할인" + 0.035*"오류" + 0.026*"하루" + 0.021*"네트워크" + 0.021*"종일" + 0.021*"연결" + 0.018*"기대

In [154]:
topic_8 = topictable[topictable['가장 비중이 높은 토픽'] == 7.0]

print('토픽 8에 해당하는 리뷰의 개수:', topic_8.shape[0], '개')

토픽 8에 해당하는 리뷰의 개수: 128 개


In [155]:
# 토픽 8에 해당하는 리뷰들만 모아서 하나의 df 만들기

topic_8_list = topic_8['리뷰 번호'].unique()
df_topic_8 = df_before.loc[topic_8_list, :]

df_topic_8.head(5)

,DATE,STAR,REVIEW,LIKE
1,2020-03-30,2,오늘 하루종일 먹통에 2중 결재 됐습니다 확인해주세요,0
5,2020-04-13,5,만족함..,1
29,2020-04-10,1,이거 진짜 뭔 앱이 연결도 안되고 오류나고 *레기인지..,0
43,2020-03-30,1,와 앱 실행 너무 안되요 너무너무너무 너어어어무!!!!!! LTE로 해도 안되고 와...,11
66,2020-04-14,4,앱로딩이 잘안됨,0


In [156]:
# 리뷰 전부 모아보기 --> 복사해서 R로 들고가서 감성분석 진행

df_topic_8['REVIEW'].unique()

array(['오늘 하루종일 먹통에 2중 결재 됐습니다 확인해주세요', '만족함..',
       '이거 진짜 뭔 앱이 연결도 안되고 오류나고 *레기인지..',
       '와 앱 실행 너무 안되요 너무너무너무 너어어어무!!!!!! LTE로 해도 안되고 와이파이는 된다고 해서 와이파이로 했더니 여전히 안되고 이건 뭐 앱을 사용 하라는 건지 말라는 건지 모르겠네요;;;ㅜㅜㅜ 부산사람들이 쓰는 앱일텐데 사용자가 많다고 하면 대체 전국민이 쓰는 앱들은 어떻게 켜지는 건가요ㅠㅠㅠ 다른 분들도 계속 안 켜진다고 건의 하시는 거 같은데 이 부분은 빠른시일내에 조치를 취해주셨으면 합니다.',
       '앱로딩이 잘안됨', '잦은 오류',
       '회원가입이 안 돼요 다음주에 만 18세 되는 사람인데요 계속 만 14세 이상만 가입이 된다고 떠요 어떡해요',
       '정말 오류가 많이 뜨네요 오류가 계속 떠서 사용하기 너무 힘들어요', '업데이트가 계속 언되어서 충전이 안됩니다',
       '고객 서비스가 최악이네요 어플 오류로 인하여 충전 하려고 하니 바탕화면으로 팅기는데 1:1 문의를 넣어도 5일동안 응답 안하고 전화는 하루에 4번씩 해도 하루종일 바빠서 못받는다고 하네요.',
       '감사합니다', '잔액 확인하려고 어플 들어가는데 접속 조차 안되면...카드를 어떻게 사용하라는건가요 대체',
       '감사합니다 혜택이 너무 좋아요 ~~',
       '아니 와이파이 네 칸 풀로뜨고 유튜브 등등 빠르게 잘되는데 왜 이 앱만들어거면 네트워크 연결이 안된다고 뜰까요..가입한 이후로 한번도 켜지지않아 카드 신청도 못하고 있습니다. 연결문제 좀 해결해주세요.',
       '평점이 왜 2점대일까 했는데 오류 진짜 많이나고 쓰레기 같아요 충전 해야되는데 어차피 오늘 안에 안고쳐질거고 하루 기다려야겠죠? 진짜 별로',
       '너무 너무 좋아요 감사합니다',
       '사람들은 10% 할인에 대해 생각은 안하고 불편함만 생각하네요. 너도 

#### 토픽별 감성분석 

- KnuSentiLex 감성사전 이용 https://github.com/park1200656/KnuSentiLex/blob/master/KnuSentiLex/SentiWord_Dict.txt
- -2, -1, 0, 1, 2 감성값에 해당하는 단어들이 있는지 보고, 각 토픽별 총 감성점수를 내는 방식
- __R에서 진행__ https://rpubs.com/shinhaenglee/767595 코드 참고

#### 워드 클라우드

- 앞서 전처리 과정에서 형태소 분석기로 명사 추출 후 불용어 제거한 단어들로 word cloud 시각화 
- 서버에서는 한글이 보이지 않는 현상 -> 주피터 노트북에서 진행하기 위해 명사들을 df 형태로 바꾼 뒤 csv 파일로 저장해서 옮기기

In [127]:
nouns[:5]

[['업데이트', '설치'],
 ['오늘', '하루', '종일', '먹통', '결재', '확인'],
 [],
 ['데이터'],
 ['카드',
  '취소',
  '금액',
  '회복',
  '문의',
  '하루',
  '종일',
  '전화',
  '연결',
  '지난번',
  '문의',
  '답변',
  '대기',
  '오늘',
  '통화',
  '연결',
  '계속',
  '포기',
  '문의',
  '다시',
  '답변',
  '도대체']]

In [128]:
# 중첩리스트 제거

noun_list = []
for i in range(len(nouns)):
    for j in range(len(nouns[i])):
        noun_list.append(nouns[i][j])

In [129]:
x = pd.DataFrame(noun_list)

x

,0
0,업데이트
1,설치
2,오늘
3,하루
4,종일
...,...
7853,계속
7854,네트워크
7855,연결
7856,불안정
